In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
           f.explode(
               f.split(lines.value, " ")
           ).alias("word")
        )

# Generate running word count
wordCounts = words.groupBy("word").count()

# Start running the query that prints the running counts to the console
# outputMode("complete")는 전체 집계 결과를 출력하도록 설정합니다.
# format("console")은 데이터를 콘솔에 출력하도록 설정합니다.
query = wordCounts \
        .writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# # append:
# # 설명: 새로운 행만 추가합니다.
# # 사용 시기: 데이터가 항상 증가하는 경우, 예를 들어 로그 데이터나 센서 데이터 스트림.
# # 제약 조건: 집계 쿼리에서 사용하기 어렵습니다.
# query = wordCounts.writeStream.outputMode("append").format("console").start()

# # complete:
# # 설명: 전체 결과 테이블을 매번 출력합니다.
# # 사용 시기: 집계 결과 전체를 매번 다시 출력해야 할 때, 예를 들어, 전체 단어 빈도 집계.
# # 제약 조건: 전체 결과를 출력하기 때문에 대용량 데이터셋에서는 비효율적일 수 있습니다.
# query = wordCounts.writeStream.outputMode("complete").format("console").start()

# # update:
# # 설명: 업데이트된 결과만 출력합니다.
# # 사용 시기: 집계 결과가 자주 업데이트되는 경우, 예를 들어, 카운트가 변경된 단어만 출력.
# # 제약 조건: 모든 쿼리에서 사용할 수 있는 것은 아닙니다. 특히, 키가 변경되지 않는 집계에서 유용합니다.
# query = wordCounts.writeStream.outputMode("update").format("console").start()

In [ ]:
# # format은 스트리밍 데이터를 어디에, 그리고 어떻게 저장할지 설정합니다. 여러 가지 형식이 있습니다:

# # console:
# # 설명: 스트리밍 데이터를 콘솔에 출력합니다.
# # 사용 시기: 디버깅 및 테스트 목적으로 사용.
# # 제약 조건: 대규모 데이터에 적합하지 않음.
# query = wordCounts.writeStream.format("console").start()

# # memory:
# # 설명: 스트리밍 데이터를 인메모리 테이블에 저장합니다.
# # 사용 시기: 쿼리 결과를 빠르게 확인하고 싶을 때.
# # 제약 조건: 작은 데이터셋에 적합.
# query = wordCounts.writeStream.format("memory").queryName("tableName").start()

# # parquet:
# # 설명: 스트리밍 데이터를 Parquet 파일 형식으로 저장합니다.
# # 사용 시기: 대규모 데이터 저장 및 후속 분석을 위해.
# # 제약 조건: HDFS 또는 다른 파일 시스템 경로 필요.
# query = wordCounts.writeStream.format("parquet").option("path", "/path/to/output").option("checkpointLocation", "/path/to/checkpoint").start()

# # csv:
# # 설명: 스트리밍 데이터를 CSV 파일 형식으로 저장합니다.
# # 사용 시기: 데이터 공유 및 이식성을 위해.
# # 제약 조건: HDFS 또는 다른 파일 시스템 경로 필요.
# query = wordCounts.writeStream.format("csv").option("path", "/path/to/output").option("checkpointLocation", "/path/to/checkpoint").start()

# # json:
# # 설명: 스트리밍 데이터를 JSON 파일 형식으로 저장합니다.
# # 사용 시기: JSON 형식이 필요한 경우.
# # 제약 조건: HDFS 또는 다른 파일 시스템 경로 필요.
# query = wordCounts.writeStream.format("json").option("path", "/path/to/output").option("checkpointLocation", "/path/to/checkpoint").start()

# # kafka:
# # 설명: 스트리밍 데이터를 Kafka 토픽으로 전송합니다.
# # 사용 시기: Kafka를 사용한 데이터 파이프라인 구축 시.
# # 제약 조건: Kafka 브로커 설정 필요.
# query = wordCounts.writeStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("topic", "topicName").start()